## Introduction
We have a large dataset on the anime watchers on the MyAnimeList website. 
In this notebook, we will explore and discuss the findings of trying to create an Anime recommendation system from the data.

## Data
In order to solve our objective, we are making use of a Kaggle dataset that essentially includes the three following things sourced from the website MyAnimeList (an online website community/database for people who watch anime):
- User List: a snapshot list of all the users in their database and some profile data, including but not limited to their birthdate, gender, location, number of anime watched, etc.
- Anime List: a snapshot list of all anime in their database when this dataset was curated
- Anime-lists List: a snapshot of the lists of anime each User has interacted with in some way (marked as to watch, watched in entirety or partly, etc.), their ratings, how many comments they left on them, etc.
There is a lot of data here to inspect.


## Data Preparation

## Modeling

## Performance Results

## Discussion

### Association Rule Mining
The association rules work pretty well, considering that it only uses the information inherent in the sets of anime that each user watches. We leverage the data in the wisdom of the crowds, assuming that most people interact with anime that they actually like on the MyAnimeList site. The way we use the association rules we find from the dataset would give the opposite effect of what we want if most of the people only interacted with anime that they deeply disliked. We would then have created an "anime to stay away from" recommender instead. If it is not explicit, this approach thus does not make any use of the score / rating that the users give to the anime that they watch.

That is were the next method comes in to try to leverage that scoring information.

### SVD

Using the SVD method, we find ...

### kNNs
When using kNNs, we needed to be wary of what kinds of features to use from the dataset. In our experimentation, we tried to recommend anime similar based off of genre and type. Recommendations on genre alone seemed pretty good, so we thought adding the type feature would also be good, but adding that in and testing it out, it was not as good just using it as a human. Since the type categories were mutually exclusive, if you had a list of TV anime, you would only be recommended TV anime. You won't be recommended any the other types like Movies or OVA. This didn't make sense from a design standpoint, so we removed it and for our comparisons between different modeling standpoints, we only used kNNs that worked on one hot encoded genre information.




## Conclusions